In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [3]:
from datasets import load_dataset

dataset = load_dataset("wikisql")

README.md:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

wikisql.py:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

The repository for wikisql contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikisql.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [4]:
dataset.keys()
dataset['train'][0]

{'phase': 1,
 'question': 'Tell me what the notes are for South Australia ',
 'table': {'header': ['State/territory',
   'Text/background colour',
   'Format',
   'Current slogan',
   'Current series',
   'Notes'],
  'page_title': '',
  'page_id': '',
  'types': ['text', 'text', 'text', 'text', 'text', 'text'],
  'id': '1-1000181-1',
  'section_title': '',
  'caption': '',
  'rows': [['Australian Capital Territory',
    'blue/white',
    'Yaa·nna',
    'ACT · CELEBRATION OF A CENTURY 2013',
    'YIL·00A',
    'Slogan screenprinted on plate'],
   ['New South Wales',
    'black/yellow',
    'aa·nn·aa',
    'NEW SOUTH WALES',
    'BX·99·HI',
    'No slogan on current series'],
   ['New South Wales',
    'black/white',
    'aaa·nna',
    'NSW',
    'CPX·12A',
    'Optional white slimline series'],
   ['Northern Territory',
    'ochre/white',
    'Ca·nn·aa',
    'NT · OUTBACK AUSTRALIA',
    'CB·06·ZZ',
    'New series began in June 2011'],
   ['Queensland',
    'maroon/white',
    'nnn·aaa

In [5]:
{
  'question': 'how many employees are in the table?',
  'sql': {
    'human_readable': 'SELECT COUNT * FROM table',
    'sel': 0,
    'agg': 4,
    'conds': []
  },
  'table_id': '1-10015132-1',
  'table': {
    'header': ['Name', 'Position', 'Number of Employees'],
    'rows': [['A Co.', 'Tech', '15'], ...]
  }
}


{'question': 'how many employees are in the table?',
 'sql': {'human_readable': 'SELECT COUNT * FROM table',
  'sel': 0,
  'agg': 4,
  'conds': []},
 'table_id': '1-10015132-1',
 'table': {'header': ['Name', 'Position', 'Number of Employees'],
  'rows': [['A Co.', 'Tech', '15'], Ellipsis]}}

In [6]:
import pandas as pd

# Convert 1000 rows to DataFrame
df = pd.DataFrame(dataset['train'][:1000])

# Show basic columns
df[['question', 'sql', 'table']].head()


,question,sql,table
0,Tell me what the notes are for South Australia,{'human_readable': 'SELECT Notes FROM table WH...,"{'header': ['State/territory', 'Text/backgroun..."
1,What is the current series where the new serie...,{'human_readable': 'SELECT Current series FROM...,"{'header': ['State/territory', 'Text/backgroun..."
2,What is the format for South Australia?,{'human_readable': 'SELECT Format FROM table W...,"{'header': ['State/territory', 'Text/backgroun..."
3,Name the background colour for the Australian ...,{'human_readable': 'SELECT Text/background col...,"{'header': ['State/territory', 'Text/backgroun..."
4,how many times is the fuel propulsion is cng?,{'human_readable': 'SELECT COUNT Fleet Series ...,"{'header': ['Order Year', 'Manufacturer', 'Mod..."


In [7]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return re.sub(r'\s+', ' ', text).strip()

df['question_clean'] = df['question'].apply(clean_text)


In [8]:
# Check for missing values
print(df.isnull().sum())

# Drop rows where 'question' or 'sql' is missing
df = df.dropna(subset=['question', 'sql'])

print(f"Remaining rows after dropping missing values: {len(df)}")


phase             0
question          0
table             0
sql               0
question_clean    0
dtype: int64
Remaining rows after dropping missing values: 1000


In [9]:
!pip install transformers

from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
text_column = df['question_clean'].tolist()

inputs = tokenizer(
    text_column,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors="pt"
)
